In [1]:
"""
#get item
*Tasks*
Features in the node dataset:
- MRE and active gene labels (DONE)
- Chromosome position (DONE)
- Sequence information (DONE)
- ChIP-seq signals (DONE)
- RNA-seq signals (DONE)
- Micro-C signals (for each p-value threshold)
    -Un-Filtered
    -Interaction Bins
    -Contact Count
    -Number of Neighbors
    
    -Filtered
    -Interaction Bins
    -Contact Count
    -Number of Neighbors

Output: datasets are stored as .pkl or .pt files

Node DataLoader Features (Chromsome Positions, Sequence Information, RNA-seq signals, ChIP-seq signals, MRE and Gene Labels
Graph DataLoader Features (Node Features: Chromosome Positions, Sequence Information, RNA-seq signals, ChIP-seq signals, MRE and Gene Labels; Edge Features: Interaction Bins, Contact Counts, P-Values, Number of Neighbors)
-look at types of graph dataloaders

*Add Additional Datasets (specify requirements)*

#generate a torch dataset (generate node matrix, generate edge matrix - binary from the microC data)
"""

'\n#get item\n*Tasks*\nFeatures in the node dataset:\n- MRE and active gene labels (DONE)\n- Chromosome position (DONE)\n- Sequence information (DONE)\n- ChIP-seq signals (DONE)\n- RNA-seq signals (DONE)\n- Micro-C signals (for each p-value threshold)\n    -Un-Filtered\n    -Interaction Bins\n    -Contact Count\n    -Number of Neighbors\n    \n    -Filtered\n    -Interaction Bins\n    -Contact Count\n    -Number of Neighbors\n\nOutput: datasets are stored as .pkl or .pt files\n\nNode DataLoader Features (Chromsome Positions, Sequence Information, RNA-seq signals, ChIP-seq signals, MRE and Gene Labels\nGraph DataLoader Features (Node Features: Chromosome Positions, Sequence Information, RNA-seq signals, ChIP-seq signals, MRE and Gene Labels; Edge Features: Interaction Bins, Contact Counts, P-Values, Number of Neighbors)\n-look at types of graph dataloaders\n\n*Add Additional Datasets (specify requirements)*\n\n#generate a torch dataset (generate node matrix, generate edge matrix - binar

In [2]:
import pandas as pd
import numpy as np
from Bio import SeqIO

/users/ssgunase/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
chipseq_dir = '/oscar/data/larschan/shared_data/bind_gps/chipseq'
labels_dir = '/oscar/data/larschan/shared_data/bind_gps/labels'
data_folder = '/oscar/data/larschan/shared_data/BindGPS/data'
#fasta file is here^
microC_dir = f'{data_folder}/S2_gfp_RNAi_significant_interactions_v2'

resolutions = ['1kb', '5kb']
pval_thresholds = ['0_001', '0_01', '0_1']
genome = SeqIO.to_dict(SeqIO.parse(f'{data_folder}/dm6.fa', "fasta"))

In [4]:
#NODE Dataset
resolutions = ['1kb', '5kb']
for resolution in resolutions:
    #rnaseq data
    rnaseq_df = pd.read_csv(f'{data_folder}/mE_RNAseq_RPKMnorm.{resolution}_binned.tsv', sep="\t")
    rnaseq_df = rnaseq_df.fillna(0) #replace all NaNs with 0
    
    rnaseq_df['DNA sequence'] = None
    #configure DNA sequence
    for index, row in rnaseq_df.iterrows():
        chrom, start, end = row['chr'], int(row['start']), int(row['end'])
        if chrom in genome:
            seq = genome[chrom].seq[start:end]
            rnaseq_df.at[index, 'DNA sequence'] = str(seq)

    #chipseq data
    chipseq_df = pd.read_csv(f'{chipseq_dir}/histone_mods_and_GA_factors.feature_matrix.{resolution}_res.tsv', sep="\t")  
    chipseq_df = chipseq_df.drop(columns=['chr', 'start', 'end'])

    #active gene labels
    gene_labels_df = pd.read_csv(f'{labels_dir}/dc_active_vs_autosome_active_gene_labels.{resolution}_res.tsv', sep="\t")
    chipseq_df['gene_labels'] = gene_labels_df['labels']

    #mre labels
    mre_labels_df = pd.read_csv(f'{labels_dir}/autosomal_mre_vs_xchr_mre_labels.{resolution}_res.tsv', sep="\t")
    chipseq_df['mre_labels'] = mre_labels_df['labels']
    
    node_df = pd.concat([rnaseq_df, chipseq_df], axis=1, join='outer')
    #reset the index
    node_dropped_df = node_df[node_df['chr'] != 'chrM'] #drop the M chromosome
    node_dropped_df = node_dropped_df.reset_index(drop=True)
    
    dir_path = "/oscar/data/larschan/shared_data/BindGPS/data/datasets"
    node_dropped_df.to_pickle(f"{dir_path}/node_dataset_{resolution}.pkl")

In [5]:
# node_dropped_df

,chr,start,end,counts,expression_level,gene_in_bin,gene_id,DNA sequence,clamp,gaf,...,h3k27me3,h3k36me3,h3k4me1,h3k4me2,h3k4me3,h3k9me3,h4k16ac,psq,gene_labels,mre_labels
0,chr2L,0,5000,0.0,0.0,False,0,Cgacaatgcacgacagaggaagcagaacagatatttagattgcctc...,0.011338,0.029695,...,0.000000,20.003200,0.0,0.000000,0.0,0.000000,0.0,30.074890,0,1
1,chr2L,5000,10000,19.0,2.0,True,FBgn0002121,TGCCTCTCATTCTGTCTTATTTTACCGCAAACCCAAatcgacaatg...,20.016980,20.031764,...,10.088679,290.071502,1.0,0.000000,0.0,20.028571,0.0,70.098778,1,1
2,chr2L,10000,15000,0.0,0.0,False,0,GAGGAGAATGCAAAAAAGCTAAGAACAAAACAATTACTACAAATCG...,10.023666,10.011295,...,0.000000,140.053661,0.0,110.071154,0.0,0.000000,0.0,70.025733,1,0
3,chr2L,15000,20000,0.0,0.0,False,0,ATTCGACGGCGGTTCTGGGTTATCTATGCTCCAAGTGGCGTATGAA...,0.072610,0.058189,...,0.000000,200.062504,0.0,130.005405,0.0,0.000000,0.0,60.001147,1,1
4,chr2L,20000,25000,0.0,1.0,True,"FBgn0031209,FBgn0263584",GTGGCCGAATTTATTCTAAACTGAAAATAATAATAAAAATTAATCA...,0.079141,0.071863,...,10.025000,160.083882,0.0,0.000000,0.0,0.000000,0.0,70.087586,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27508,chrY,3645000,3650000,0.0,1.0,True,FBgn0267592,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0.000641,0.000349,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0,0
27509,chrY,3650000,3655000,0.0,0.0,False,0,GTTCTCCACACAAAAAGAATTTTTTCATATACCCTATATAAACGAA...,0.059530,0.050137,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,10.009552,0,1
27510,chrY,3655000,3660000,0.0,0.0,False,0,acacggagtaaaaatccgcccagtttgcttagcctccgccaaacgt...,0.000308,0.000322,...,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.0,0.017110,0,1
27511,chrY,3660000,3665000,0.0,0.0,False,0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0.000460,0.000196,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0,0


In [8]:
#EDGE Dataset
# chromosome M is not in there
resolutions = ['1kb', '5kb']
pval_thresholds = ['0_001', '0_01', '0_1']
for resolution in resolutions:
    for pval in pval_thresholds:
        microc_df = pd.read_table(f'{microC_dir}/S2_gfpi_loops.{resolution}_res.p{pval}.v2.bedpe')
        edges = microc_df.loc[:,'bin1':'bin2']
        edge_weight = -1 * np.log10(microc_df['p-value'] + 1e-300)
        max = np.percentile(edge_weight, q=99.99) #99% maximum - set that as the maximum to account for outliers
        edge_weight = np.minimum(edge_weight, max)
        microc_df['p-value_transformed'] = edge_weight

        dir_path = "/oscar/data/larschan/shared_data/BindGPS/data/datasets"
        microc_df.to_pickle(f"{dir_path}/edge_dataset_{pval}_{resolution}.pkl")

In [13]:
print(microc_df['chr1'].unique())

['chr2L' 'chr2R' 'chr3L' 'chr3R' 'chr4' 'chrX' 'chrY']


In [44]:
seq = genome['chrY'].seq[3663000:3664000] #look into this
print(seq)

NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAACTCCTCGTTGTATGTTCCGATACAAGCGATTGATGTTTGGTGTCAACGCCGCACCCGAAATCTTTCAACGAAGACTTCAGGAGCTTTATATAGCGTGTTCTAATGTCTTGAACTACATTGATGACATCATTGTATTTGGAACAATGAAAATGATCCCGATAACGCGGTCAAGGAAGTCAAAAAGATAGTAAAGGATAATAATATTCTCATGAATGCGGAAAAATGAATTGAGGTTGATACGGAAAAAGTAGCAACAAAAAAGGACCGGAAATTCAATTGGAGTGAAGCAGAGAGAAAAGCTTTCGTGTGTCTTAAGTCTTGTTTTTCAGGATTTGCGACAGATTCCGTTCCCCCAACTGTAATAGCTTGACCGTCCCGGTTGTTCTCCAAAACATTTTTCGTATTACCCTCGCCCGGACCTTAGAAACAATCCGGCCATGGTGTGCGCTAGCAGCGCCAAATAAACATAGAAATCAGTACACAATATAATACATTAATCGATTAAGCTATTTAGCGCAGTACGTCCTCTTCAAGATTTTCAACTGTTTTATGTGTAAATATTTTCCTATATCGATTTTTGAAGAGCGGTATGAATCAGTTTTTTAGCTTTATCTTGGGTACTTAAATTAAGATAGACAAAACTAATAATTATTTTAAATATTTATTGCAGGTTGTGGTTTGTGAACTTAACCGACGGTATTGCAATCAGCATGGCTCG

In [ ]:
# unique_chr = node_df['chr'].unique()
# unique_chr